In [ ]:
import sklearn
from sklearn.datasets import fetch_openml
import random
import numpy as np
mnist = fetch_openml('mnist_784')


In [ ]:

def selectnumbers(data, target):
  numbers = random.sample(range(0,70000),10000) 
  data = data[numbers]
  target = target[numbers]
  return data, target

def train_test_split(data,label,k=4):
  size = len(data)//4 # -> len(data) : 10000 -> size = 2500
  trains = []
  validations = []
  for i in range(k):
    validation_data = data[i*size:(i*size)+size]
    validation_label = label[i*size:(i*size)+size]
    train_data = np.concatenate((data[:i*size], data[(i*size)+size:]), axis=0)
    train_label = np.concatenate((label[:i*size], label[(i*size)+size:]), axis=0)
    trains.append((train_data, train_label))
    validations.append((validation_data, validation_label))

  # k=0 : 0~2500 valdation(test), 2501~10000 train
  # k=1 : 2501~5000 validation(test), 0~2500, 5001~10000 train
  return trains, validations

def selectdata(data,target,mode):
  numbers = []
  if mode=='binary':
    while len(numbers)<2:
      number = int(np.random.randint(0,10,1))
      if number not in numbers:
        numbers.append(number)
  elif mode=='ternary':
    while len(numbers)<3:
      number = int(np.random.randint(0,10,1))
      if number not in numbers:
        numbers.append(number)
  indexes = []
  for number in numbers:
    indexes += np.where(target==number)[0].tolist()
  return data[indexes], target[indexes]


In [ ]:
from sklearn.svm import SVC, LinearSVC
import matplotlib.pyplot as plt
### error 출력 제거 ###
import warnings
from sklearn.exceptions import ConvergenceWarning

def accuracy_score(preds, targets):
  return sum(preds==targets)/len(targets)

def SVM(train_dataset, validation_dataset, mode, k=4):
  
  svm = SVC(max_iter=1000, kernel='linear')
  for i in range(k):
    print('===========================================================')
    train_x, train_y = train_dataset[i]
    print('select label is ', np.unique(train_y))
    validation_x, validation_y = validation_dataset[i]
    svm.fit(train_x, train_y)
    
    w = np.linalg.norm(svm.coef_,axis=1)
    b = svm.intercept_
    ### w와 b 출력 ###
    #print('w = ', w)
    #print('b = ', b)
    x_axis = np.arange(-30,30)
    ### plot ###
    for _w, _b in zip(w,b):
        y_axis = _w*x_axis + _b
        print('decision equation : y = {:.5f}x + {:.5f}'.format(_w,_b))
        plt.plot(x_axis,y_axis)
    plt.show()
    preds = svm.predict(validation_x)
    score = accuracy_score(validation_y, preds)
    print('K-FOLD : k={} \t score : {}'.format(i,score))

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)
    mode = 'binary'
    data = mnist['data']
    target = np.array(list(map(int, mnist['target'])))
    data, target = selectnumbers(data, target)
    data, target = selectdata(data, target, mode)
    train_dataset, validation_dataset = train_test_split(data,target)
    SVM(train_dataset, validation_dataset, mode)